In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('AllInOneCustom.xlsx')

In [7]:
df['CGM_Data'] = df['CGM (mg / dl)'].fillna(df['CGM '])

In [8]:
df.drop(['CGM (mg / dl)', 'CGM '], axis=1, inplace=True)

In [9]:
df.to_excel('AllInOneCustom.xlsx', index=False)

In [5]:
df["Date"] = pd.to_datetime(df["Date"])

In [6]:
df["Date"] = pd.to_datetime(df["Date"])
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Day"] = df["Date"].dt.day
df["Hour"] = df["Date"].dt.hour
df["Minute"] = df["Date"].dt.minute
df["Second"] = df["Date"].dt.second
df["AM/PM"] = df["Date"].dt.strftime("%p")

In [7]:
df.to_excel('AllInOneCustom.xlsx', index=False)